In [2]:
# Load basic packages
import numpy as np
import pandas as pd

import autograd.numpy as numpy
import autograd.numpy.random as npr

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn import metrics
from sklearn import model_selection

# Useful for saving our models.
import pickle
print("Imported all!")

df = pd.read_csv('diabetes.csv')
df = df.dropna()
diabetes_numpy = df.values
print("Shape of the loaded dataset: {} rows and {} columns.".format(diabetes.shape[0], diabetes.shape[1]))

Imported all!
Shape of the loaded dataset: 253680 rows and 22 columns.
